In [1]:
import pandas as pd
from pandas.io.gbq import read_gbq
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## The general challenge is to look at how extreme weather affects 311 service requests: First we have to define and identify extreme weather. Second we have to look how extreme weather events affect 311 request types and volume. Lastly, can we predict call/complaint volume for extreme weather events such as high/low temperature, snow, rain etc. ?

In [2]:
#let's look at the data! start with weather data: pull for 2010-2017 for the central park station
query = "SELECT DATE(CAST(year AS INT64), CAST(mo AS INT64), CAST(da AS INT64)) as created_date, temp, wdsp, mxpsd, gust, max, min, prcp, sndp, snow_ice_pellets FROM `bigquery-public-data.noaa_gsod.gsod20*` WHERE _TABLE_SUFFIX BETWEEN '10' AND '17' AND stn = '725053' AND wban = '94728'"
weather = read_gbq(query=query, project_id='opendataproject-180502', dialect='standard')

Requesting query... ok.
Query running...
Query done.
Processed: 2.6 Gb

Retrieving results...
Got 1726 rows.

Total time taken 3.9 s.
Finished at 2017-09-23 17:13:03.


pd.set_option("display.max_columns",1001)

In [3]:
weather.head()

,created_date,temp,wdsp,mxpsd,gust,max,min,prcp,sndp,snow_ice_pellets
0,2015-12-25,62.0,0.7,4.1,999.9,72.0,57.0,0.01,999.9,0
1,2017-09-12,69.0,0.8,4.1,999.9,78.1,55.0,0.00,999.9,0
2,2015-09-06,74.1,1.0,4.1,999.9,87.1,64.9,0.00,999.9,0
3,2015-09-16,75.8,1.5,4.1,999.9,88.0,62.1,0.00,999.9,0
4,2013-08-17,72.6,1.6,4.1,999.9,84.0,64.0,0.00,999.9,0


In [7]:
#do some aggregating for simplicty
query = """SELECT DATE(created_date) AS `created_date`, complaint_type, count(unique_key) AS `count_n`
FROM `bigquery-public-data.new_york.311_service_requests`
GROUP BY DATE(created_date), complaint_type"""
dfcounts = read_gbq(query=query, project_id='opendataproject-180502', dialect='standard')

Requesting query... ok.
Query running...
Query done.
Processed: 517.8 Mb

Retrieving results...
  Got page: 1; 28% done. Elapsed 14.51 s.
  Got page: 2; 55% done. Elapsed 18.95 s.
  Got page: 3; 83% done. Elapsed 24.2 s.
  Got page: 4; 100% done. Elapsed 27.66 s.
Got 362184 rows.

Total time taken 29.75 s.
Finished at 2017-09-23 17:27:05.


In [8]:
dfcounts.head()

,created_date,complaint_type,count_n
0,2010-08-06,Overgrown Tree/Branches,61
1,2010-04-01,Traffic/Illegal Parking,61
2,2017-02-17,Benefit Card Replacement,61
3,2016-11-04,Benefit Card Replacement,61
4,2015-02-13,Benefit Card Replacement,61


In [10]:
dfcounts.sort_values(['created_date'], ascending = 0).head()

,created_date,complaint_type,count_n
99176,2017-09-22,Illegal Parking,21
289674,2017-09-22,School Maintenance,1
295959,2017-09-22,Street Sign - Damaged,1
194626,2017-09-22,Rodent,5
159377,2017-09-22,Noise - Street/Sidewalk,58
